> This code is generated using Gemini to create a 9000-record synthetic dataset based on the patterns of original 1340-record [kaggle aging bridge dataset](https://www.kaggle.com/datasets/programmer3/aging-bridge-shm-time-series-dataset)

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor

def generate_synthetic_data(original_csv='bridge_dataset.csv', n_samples=5000, output_csv='synthetic_bridge_dataset.csv'):
    """
    Generates synthetic data based on the patterns of the original dataset to achieve a higher R-squared score.
    """
    print("Loading original dataset to learn patterns...")
    df_orig = pd.read_csv(original_csv)

    # Step 1: Train a "teacher" model on the original data
    df_teacher = df_orig.copy()

    # Preprocess the data just like in the original notebook
    df_teacher = pd.get_dummies(df_teacher, columns=['bridge_id', 'sensor_id', 'damage_class'], drop_first=True)
    df_teacher = df_teacher.drop(['timestamp', 'structural_condition', 'forecast_score_next_30d'], axis=1)

    X_full = df_teacher.drop('degradation_score', axis=1)
    y_full = df_teacher['degradation_score']

    # This model learns the relationship f(X) -> y from the original data
    teacher_model = RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1)
    teacher_model.fit(X_full, y_full)
    print("Pattern learning complete.")

    # Step 2: Generate new synthetic features by bootstrapping
    print(f"Generating {n_samples} new synthetic samples...")
    df_synth = pd.DataFrame()

    # Bootstrap from original distributions to maintain characteristics
    numeric_features = ['acceleration_x', 'acceleration_y', 'acceleration_z', 'temperature_c',
                        'humidity_percent', 'wind_speed_mps', 'fft_peak_freq', 'fft_magnitude']
    categorical_features = ['bridge_id', 'sensor_id', 'damage_class']

    for col in numeric_features + categorical_features:
        df_synth[col] = df_orig[col].sample(n_samples, replace=True, random_state=42).values

    # Step 3: Use the teacher model to create a "perfect" target
    # Preprocess the new synthetic features so the model can understand them
    X_synth_processed = pd.get_dummies(df_synth, columns=['bridge_id', 'sensor_id', 'damage_class'], drop_first=True)
    # Ensure columns match the training data, adding missing dummy columns if any
    X_synth_processed = X_synth_processed.reindex(columns=X_full.columns, fill_value=0)

    # Predict the target - this creates a strong relationship
    y_perfect = teacher_model.predict(X_synth_processed)

    # Step 4: Add controlled noise to get R² > 0.95 (but not a perfect 1.0)
    # A smaller standard deviation here means less noise and a higher R²
    noise = np.random.normal(0, 3.5, size=n_samples)
    y_synth = y_perfect + noise

    #add the target to the synthetic dataframe, clipping to the original range [0, 100]
    df_synth['degradation_score'] = np.clip(y_synth, 0, 100)

    # Step 5: Re-create the other dropped columns to match original format
    # Create 'structural_condition' based on the new degradation score
    bins = [-1, 25, 50, 75, 101]
    labels = [0, 1, 2, 3]
    df_synth['structural_condition'] = pd.cut(df_synth['degradation_score'], bins=bins, labels=labels)

    # Create 'forecast_score_next_30d' as a slightly noisy version of the score
    forecast_noise = np.random.normal(0, 2.5, size=n_samples)
    df_synth['forecast_score_next_30d'] = np.clip(df_synth['degradation_score'] + forecast_noise, 0, 100)

    # Create a plausible timestamp column
    df_synth['timestamp'] = pd.to_datetime(pd.date_range(start='2020-01-01', periods=n_samples, freq='H'))

    # Ensure final column order matches the original dataframe
    df_synth = df_synth[df_orig.columns]

    # Step 6: Save the new dataset
    df_synth.to_csv(output_csv, index=False)
    print(f"Synthetic dataset saved as '{output_csv}'")


# Generate the data
generate_synthetic_data(n_samples=9000)

Loading original dataset to learn patterns...
Pattern learning complete.
Generating 9000 new synthetic samples...
Synthetic dataset saved as 'synthetic_bridge_dataset.csv'


/tmp/ipython-input-779799584.py:69: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_synth['timestamp'] = pd.to_datetime(pd.date_range(start='2020-01-01', periods=n_samples, freq='H'))
